# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [4]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

Configurando a biblioteca:

In [5]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [6]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [7]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [8]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [9]:
def simulate_single_flow(g, s, t, whichSuccessorGo, shouldKeep):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
        
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0
    
    funcs = [random_geodesic_successor,random_path_successor,random_trail_successor, random_walk_successor ]

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Deixa de ser dono do insumo.
            if(not shouldKeep):
                g.nodes[n]['owner'] = False

            # Escolhe aleatoriamente um dos sucessores.
            try:
                m = funcs[whichSuccessorGo-1](g, n)
            except:
                continue
                

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [10]:
def simulate_successful_flow(g, s, t, whichSuccessorGo, shouldKeep):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, whichSuccessorGo, shouldKeep)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [11]:
def simulate_all_flows(g,whichSuccessorGo, shouldKeep):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, whichSuccessorGo, shouldKeep)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

In [12]:
cc = nx.closeness_centrality(g)

bc = nx.betweenness_centrality(g)

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

In [13]:
def simulate(g,whichFunc, shouldKeep, TIMES):

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, whichFunc, shouldKeep)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES

    return pd.DataFrame({
        'família': [g.nodes[n]['label'] for n in g.nodes],
        'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
        'closeness analítico': [cc[n] for n in g.nodes],
        'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
        'betweenness analítico': [bc[n] for n in g.nodes],
    })
    

In [14]:
a = simulate(g,1, True, 100)
a

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.333333,ginori
1,0.000000,0.000000,0.325581,0.325581,lambertes
2,0.212454,0.574615,0.482759,0.482759,albizzi
3,0.260073,0.687253,0.466667,0.466667,guadagni
4,0.000000,0.000000,0.285714,0.285714,pazzi
5,0.142857,0.406593,0.388889,0.388889,salviati
6,0.521978,1.388242,0.560000,0.560000,medici
7,0.091575,0.240879,0.482759,0.482759,tornabuon
8,0.120879,0.315220,0.400000,0.400000,bischeri
9,0.086081,0.247363,0.482759,0.482759,ridolfi


In [15]:
b = simulate(g,1, False, 100)
b

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.333333,ginori
1,0.000000,0.000000,0.325581,0.325581,lambertes
2,0.212454,0.210824,0.482759,0.482759,albizzi
3,0.260073,0.258242,0.466667,0.466667,guadagni
4,0.000000,0.000000,0.285714,0.285714,pazzi
5,0.142857,0.142857,0.388889,0.388889,salviati
6,0.521978,0.520385,0.560000,0.560000,medici
7,0.091575,0.090000,0.482759,0.482759,tornabuon
8,0.120879,0.120604,0.400000,0.400000,bischeri
9,0.086081,0.089011,0.482759,0.482759,ridolfi


In [16]:
c = simulate(g,2, True, 100)
c

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.391044,0.333333,0.230467,ginori
1,0.000000,0.397802,0.325581,0.234361,lambertes
2,0.212454,0.613022,0.482759,0.255524,albizzi
3,0.260073,0.743736,0.466667,0.264366,guadagni
4,0.000000,0.258681,0.285714,0.204411,pazzi
5,0.142857,0.435055,0.388889,0.225896,salviati
6,0.521978,0.819286,0.560000,0.268496,medici
7,0.091575,0.681429,0.482759,0.280791,tornabuon
8,0.120879,0.650495,0.400000,0.266518,bischeri
9,0.086081,0.703791,0.482759,0.279111,ridolfi


In [17]:
d = simulate(g,2, False, 100)
d

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.204666,ginori
1,0.000000,0.000000,0.325581,0.213056,lambertes
2,0.212454,0.233956,0.482759,0.253047,albizzi
3,0.260073,0.414121,0.466667,0.270372,guadagni
4,0.000000,0.000000,0.285714,0.201626,pazzi
5,0.142857,0.142857,0.388889,0.241438,salviati
6,0.521978,0.626264,0.560000,0.311041,medici
7,0.091575,0.335440,0.482759,0.258130,tornabuon
8,0.120879,0.371923,0.400000,0.249214,bischeri
9,0.086081,0.363022,0.482759,0.253833,ridolfi


In [18]:
e = simulate(g,3, True, 100)
e

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.371264,0.333333,0.222115,ginori
1,0.000000,0.388352,0.325581,0.228298,lambertes
2,0.212454,0.685495,0.482759,0.248031,albizzi
3,0.260073,0.886978,0.466667,0.256000,guadagni
4,0.000000,0.263242,0.285714,0.201152,pazzi
5,0.142857,0.422033,0.388889,0.219488,salviati
6,0.521978,1.259011,0.560000,0.257551,medici
7,0.091575,0.922527,0.482759,0.263230,tornabuon
8,0.120879,0.783187,0.400000,0.250244,bischeri
9,0.086081,0.918297,0.482759,0.262404,ridolfi


In [19]:
f = simulate(g,3, False, 100)
f

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.217994,ginori
1,0.000000,0.000000,0.325581,0.205100,lambertes
2,0.212454,0.264176,0.482759,0.268648,albizzi
3,0.260073,0.407747,0.466667,0.266886,guadagni
4,0.000000,0.000000,0.285714,0.194275,pazzi
5,0.142857,0.142857,0.388889,0.244098,salviati
6,0.521978,0.702967,0.560000,0.285141,medici
7,0.091575,0.316209,0.482759,0.242965,tornabuon
8,0.120879,0.340385,0.400000,0.230759,bischeri
9,0.086081,0.328516,0.482759,0.248642,ridolfi


In [20]:
h = simulate(g,4, True, 100)
h

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.814066,0.333333,0.142301,ginori
1,0.000000,0.701264,0.325581,0.144130,lambertes
2,0.212454,3.074505,0.482759,0.156711,albizzi
3,0.260073,4.555275,0.466667,0.160873,guadagni
4,0.000000,0.770659,0.285714,0.129011,pazzi
5,0.142857,1.805330,0.388889,0.142128,salviati
6,0.521978,6.962473,0.560000,0.170811,medici
7,0.091575,2.412967,0.482759,0.166100,tornabuon
8,0.120879,2.795165,0.400000,0.158794,bischeri
9,0.086081,2.558901,0.482759,0.165428,ridolfi


In [21]:
t = simulate(g,4, False, 100)
t

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.746593,0.333333,0.036897,ginori
1,0.000000,0.755824,0.325581,0.037200,lambertes
2,0.212454,2.464286,0.482759,0.037358,albizzi
3,0.260073,3.304505,0.466667,0.036429,guadagni
4,0.000000,0.707802,0.285714,0.038966,pazzi
5,0.142857,1.565989,0.388889,0.038310,salviati
6,0.521978,5.086538,0.560000,0.035156,medici
7,0.091575,2.495330,0.482759,0.035927,tornabuon
8,0.120879,2.476429,0.400000,0.035729,bischeri
9,0.086081,2.469670,0.482759,0.033049,ridolfi


# Análisando se os valores encontrados podem ser usados para estudo, afinal,  foram gerados a partir de uma média de 100 vezes, vai que representa algo aleatório 

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import math

In [23]:
def get_data_for_test(g,whichFunc, shouldKeep):

    for n in g.nodes:
        g.nodes[n]['historic_closeness'] = []
        g.nodes[n]['historic_betweenness'] = []

    for _ in range(10):
        simulate_all_flows(g, whichFunc, shouldKeep)

        for n in g.nodes:
            g.nodes[n]['historic_closeness'].append(g.nodes[n]['closeness'])
            g.nodes[n]['historic_betweenness'].append(g.nodes[n]['betweenness'])

    listToReturn = []
    for n in g.nodes:
        listToReturn.append([g.nodes[n]['historic_closeness'], g.nodes[n]['historic_betweenness']])
        
    return listToReturn


def tests_for_a_df(df, g, whichFunc, shouldKeep):
    
    data = get_data_for_test(g, whichFunc, shouldKeep)
    for i in range(0,len(df["betweenness simulado"])):
        teste_b = stats.ttest_1samp(a= data[i][1], popmean= df["betweenness simulado"][i])
        teste_c =  stats.ttest_1samp(a= data[i][0], popmean= df["closeness simulado"][i])
        print("Nó {0}, betweenness test: {1}".format(i, teste_b))
        print("Nó {0}, closeness test: {1}".format(i, teste_c))
        print("\n")
        

In [24]:
tests_for_a_df(a,g,1, True)
tests_for_a_df(b,g,1, False)
tests_for_a_df(c,g,2, True)
tests_for_a_df(d,g,2, False)
tests_for_a_df(e,g,3, True)
tests_for_a_df(f,g,3, False)
tests_for_a_df(h,g,4, True)
tests_for_a_df(t,g,4, False)

Nó 0, betweenness test: Ttest_1sampResult(statistic=nan, pvalue=nan)
Nó 0, closeness test: Ttest_1sampResult(statistic=23.999999999999996, pvalue=1.808926858703441e-09)


Nó 1, betweenness test: Ttest_1sampResult(statistic=nan, pvalue=nan)
Nó 1, closeness test: Ttest_1sampResult(statistic=-inf, pvalue=0.0)


Nó 2, betweenness test: Ttest_1sampResult(statistic=-1.272386535302844, pvalue=0.23512406517616796)
Nó 2, closeness test: Ttest_1sampResult(statistic=-47.99999999999999, pvalue=3.704869802185669e-12)


Nó 3, betweenness test: Ttest_1sampResult(statistic=-0.9053145896390102, pvalue=0.38889121490034007)
Nó 3, closeness test: Ttest_1sampResult(statistic=-inf, pvalue=0.0)


Nó 4, betweenness test: Ttest_1sampResult(statistic=nan, pvalue=nan)
Nó 4, closeness test: Ttest_1sampResult(statistic=inf, pvalue=0.0)


Nó 5, betweenness test: Ttest_1sampResult(statistic=nan, pvalue=nan)
Nó 5, closeness test: Ttest_1sampResult(statistic=inf, pvalue=0.0)


Nó 6, betweenness test: Ttest_1sampResult

Nó 0, betweenness test: Ttest_1sampResult(statistic=nan, pvalue=nan)
Nó 0, closeness test: Ttest_1sampResult(statistic=-1.6692460226847323, pvalue=0.1294102548987216)


Nó 1, betweenness test: Ttest_1sampResult(statistic=nan, pvalue=nan)
Nó 1, closeness test: Ttest_1sampResult(statistic=-0.21907801907234573, pvalue=0.8314769197294658)


Nó 2, betweenness test: Ttest_1sampResult(statistic=2.029192356978954, pvalue=0.07302966665735224)
Nó 2, closeness test: Ttest_1sampResult(statistic=2.1434717644076144, pvalue=0.060677656545427706)


Nó 3, betweenness test: Ttest_1sampResult(statistic=-0.8607447815394175, pvalue=0.4117217269330432)
Nó 3, closeness test: Ttest_1sampResult(statistic=1.7037711742633068, pvalue=0.12262237511577843)


Nó 4, betweenness test: Ttest_1sampResult(statistic=nan, pvalue=nan)
Nó 4, closeness test: Ttest_1sampResult(statistic=-0.523197449050258, pvalue=0.6134727121708576)


Nó 5, betweenness test: Ttest_1sampResult(statistic=inf, pvalue=0.0)
Nó 5, closeness test: Tt

Nó 0, betweenness test: Ttest_1sampResult(statistic=-1.498317871012432, pvalue=0.16827890572152876)
Nó 0, closeness test: Ttest_1sampResult(statistic=0.058686271562082296, pvalue=0.9544843839525159)


Nó 1, betweenness test: Ttest_1sampResult(statistic=-0.8191371706184071, pvalue=0.43386247234700204)
Nó 1, closeness test: Ttest_1sampResult(statistic=-2.3074854732976045, pvalue=0.0464271766502317)


Nó 2, betweenness test: Ttest_1sampResult(statistic=-1.3641121359606243, pvalue=0.2056686868984263)
Nó 2, closeness test: Ttest_1sampResult(statistic=-0.8340200962618897, pvalue=0.4258512408992321)


Nó 3, betweenness test: Ttest_1sampResult(statistic=-0.4743382309386398, pvalue=0.6465559857209984)
Nó 3, closeness test: Ttest_1sampResult(statistic=0.5755011813296438, pvalue=0.5790549909367526)


Nó 4, betweenness test: Ttest_1sampResult(statistic=0.7070118121215102, pvalue=0.49745527064186024)
Nó 4, closeness test: Ttest_1sampResult(statistic=-3.0733049953051372, pvalue=0.013282863777590862)

## Analise de erro entre Simulações e Analíticas

In [25]:
def getErrors(df):
    result_b = np.abs(df["betweenness analítico"] - df["betweenness simulado"])
    result_b = np.mean(result_b)
    result_c =  np.abs(df["closeness analítico"] - df["closeness simulado"])
    result_c =  np.mean(result_c)
    print("Média de erro entre simulado e analítico (betweenness: {0})".format(result_b))
    print("Média de erro entre simulado e analítico (closeness: {0})".format(result_c))
    print("\n")
    return result_b,result_c

In [26]:
getErrors(a)
getErrors(b)
getErrors(c)
getErrors(d)
getErrors(e)
getErrors(f)
getErrors(h)
getErrors(t)

Média de erro entre simulado e analítico (betweenness: 0.19571428571428573)
Média de erro entre simulado e analítico (closeness: 4.51490696680897e-16)


Média de erro entre simulado e analítico (betweenness: 0.000986568986569004)
Média de erro entre simulado e analítico (closeness: 4.51490696680897e-16)


Média de erro entre simulado e analítico (betweenness: 0.44947619047619053)
Média de erro entre simulado e analítico (closeness: 0.15922433481979695)


Média de erro entre simulado e analítico (betweenness: 0.13269597069597064)
Média de erro entre simulado e analítico (closeness: 0.16888872970362495)


Média de erro entre simulado e analítico (betweenness: 0.5621282051282049)
Média de erro entre simulado e analítico (closeness: 0.16999489430834833)


Média de erro entre simulado e analítico (betweenness: 0.13095238095238096)
Média de erro entre simulado e analítico (closeness: 0.1713575299595334)


Média de erro entre simulado e analítico (betweenness: 2.288186813186813)
Média de erro

(1.9532234432234428, 0.3755241804520289)

## Análise de intensidade de importância para diferentes funções (betweenness)

In [27]:
print('Geodésicas clonando: ')
print(np.mean(a["betweenness simulado"]))
print('Geodésicas transferindo: ')
print(np.mean(b["betweenness simulado"]))
print("\n")

print('Caminho clonando: ')
print(np.mean(c["betweenness simulado"]))
print('Caminho transferindo: ')
print(np.mean(d["betweenness simulado"]))
print("\n")

print('Trilha clonando: ')
print(np.mean(e["betweenness simulado"]))
print('Trilha transferindo: ')
print(np.mean(f["betweenness simulado"]))
print("\n")

print('Passeio clonando: ')
print(np.mean(h["betweenness simulado"]))
print('Passeio transferindo: ')
print(np.mean(t["betweenness simulado"]))
print("\n")

Geodésicas clonando: 
0.3114652014652014
Geodésicas transferindo: 
0.11575091575091569


Caminho clonando: 
0.5652271062271061
Caminho transferindo: 
0.2484468864468864


Trilha clonando: 
0.6778791208791208
Trilha transferindo: 
0.2467032967032967


Passeio clonando: 
2.4039377289377284
Passeio transferindo: 
2.0689743589743586




## Análise de ordem importância para diferentes funções

Por enquanto analisando esses resultados percebe-se que manter o pacote influenciou mais na disparidade dos resultados em relação ao betweness do que ao closeness. Vamos analisar agora se a sequência de "importância" mantêm-se.

In [28]:
def generateAnalisys(a):

    listaBA = a["betweenness analítico"]
    sortedListBA = []
    for i in range(0,len(listaBA)):
        sortedListBA.append((i,listaBA[i]))

    sortedListBA = sorted(sortedListBA, key=lambda x: x[1], reverse=True)
    #print(sortedListBA)
    
    listaBS = a["betweenness simulado"]

    sortedListBS = []
    for i in range(0,len(listaBS)):
        sortedListBS.append((i,listaBS[i]))

    sortedListBS = sorted(sortedListBS, key=lambda x: x[1], reverse=True)
    #print(sortedListBS)

    listaCA = a["closeness analítico"]
    sortedListCA = []
    for i in range(0,len(listaCA)):
        sortedListCA.append((i,listaCA[i]))

    sortedListCA = sorted(sortedListCA, key=lambda x: x[1], reverse=True)
    #print(sortedListCA)

    listaCS = a["closeness simulado"]
    sortedListCS = []
    for i in range(0,len(listaCS)):
        sortedListCS.append((i,listaCS[i]))

    sortedListCS = sorted(sortedListCS, key=lambda x: x[1], reverse=True)
    #print(sortedListCS)
    
    return(sortedListBA, sortedListBS, sortedListCA, sortedListCS)

In [29]:
BAa, BSa, CAa, CSa = generateAnalisys(a)
print("Betweenness Analítico")
print(BAa)
print("\n")
print("Betweenness Simulado")
print(BSa)
print("\n")
print("Closeness Analítico")
print(CAa)
print("\n")
print("Closeness Simulado")
print(CSa)


Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 1.3882417582417579), (3, 0.6872527472527474), (2, 0.5746153846153849), (5, 0.4065934065934066), (13, 0.3173626373626374), (8, 0.3152197802197803), (9, 0.24736263736263742), (7, 0.24087912087912083), (14, 0.2304945054945052), (11, 0.20774725274725256), (12, 0.05620879120879113), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333), (1,

In [30]:
BAb, BSb, CAb, CSb = generateAnalisys(b)
print("Betweenness Analítico")
print(BAb)
print("\n")
print("Betweenness Simulado")
print(BSb)
print("\n")
print("Closeness Analítico")
print(CAb)
print("\n")
print("Closeness Simulado")
print(CSb)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.5203846153846158), (3, 0.2582417582417584), (2, 0.21082417582417556), (5, 0.1428571428571427), (8, 0.12060439560439537), (13, 0.11516483516483492), (7, 0.08999999999999986), (9, 0.08901098901098888), (14, 0.08868131868131854), (11, 0.07879120879120868), (12, 0.021703296703296678), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333)

In [31]:
BAc, BSc, CAc, CSc = generateAnalisys(c)
print("Betweenness Analítico")
print(BAc)
print("\n")
print("Betweenness Simulado")
print(BSc)
print("\n")
print("Closeness Analítico")
print(CAc)
print("\n")
print("Closeness Simulado")
print(CSc)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.8192857142857144), (3, 0.7437362637362638), (9, 0.7037912087912087), (11, 0.6815384615384618), (7, 0.6814285714285717), (8, 0.6504945054945059), (2, 0.6130219780219784), (14, 0.6082417582417587), (13, 0.5779670329670331), (12, 0.5196153846153847), (5, 0.43505494505494446), (1, 0.3978021978021976), (10, 0.3967032967032966), (0, 0.39104395604395586), (4, 0.25868131868131883)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.

In [32]:
BAd, BSd, CAd, CSd = generateAnalisys(d)
print("Betweenness Analítico")
print(BAd)
print("\n")
print("Betweenness Simulado")
print(BSd)
print("\n")
print("Closeness Analítico")
print(CAd)
print("\n")
print("Closeness Simulado")
print(CSd)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.6262637362637364), (3, 0.41412087912087897), (14, 0.3830219780219779), (8, 0.37192307692307686), (11, 0.3689010989010987), (9, 0.36302197802197794), (7, 0.3354395604395604), (12, 0.24989010989010985), (13, 0.23730769230769222), (2, 0.23395604395604383), (5, 0.1428571428571427), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333), (

In [33]:
BAe, BSe, CAe, CSe = generateAnalisys(e)
print("Betweenness Analítico")
print(BAe)
print("\n")
print("Betweenness Simulado")
print(BSe)
print("\n")
print("Closeness Analítico")
print(CAe)
print("\n")
print("Closeness Simulado")
print(CSe)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 1.2590109890109893), (7, 0.9225274725274718), (9, 0.9182967032967031), (11, 0.8871428571428568), (3, 0.8869780219780214), (14, 0.8265934065934062), (8, 0.7831868131868134), (2, 0.6854945054945059), (13, 0.5973626373626376), (12, 0.5874725274725278), (5, 0.4220329670329668), (1, 0.388351648351648), (0, 0.37126373626373604), (10, 0.3692307692307692), (4, 0.2632417582417583)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.368

In [34]:
BAf, BSf, CAf, CSf = generateAnalisys(f)
print("Betweenness Analítico")
print(BAf)
print("\n")
print("Betweenness Simulado")
print(BSf)
print("\n")
print("Closeness Analítico")
print(CAf)
print("\n")
print("Closeness Simulado")
print(CSf)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.702967032967033), (3, 0.4077472527472525), (14, 0.3747252747252746), (8, 0.3403846153846154), (9, 0.3285164835164835), (11, 0.32483516483516495), (7, 0.3162087912087912), (13, 0.27192307692307705), (2, 0.26417582417582425), (12, 0.22620879120879117), (5, 0.1428571428571427), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333), (1, 

In [35]:
BAh, BSh, CAh, CSh = generateAnalisys(h)
print("Betweenness Analítico")
print(BAh)
print("\n")
print("Betweenness Simulado")
print(BSh)
print("\n")
print("Closeness Analítico")
print(CAh)
print("\n")
print("Closeness Simulado")
print(CSh)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 6.962472527472524), (3, 4.555274725274725), (14, 3.1339010989011005), (2, 3.074505494505493), (8, 2.795164835164835), (11, 2.742252747252747), (9, 2.5589010989010994), (7, 2.412967032967033), (5, 1.8053296703296695), (12, 1.6601098901098907), (13, 1.4899999999999998), (0, 0.814065934065934), (4, 0.7706593406593407), (1, 0.7012637362637366), (10, 0.5821978021978026)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526

In [36]:
BAt, BSt, CAt, CSt = generateAnalisys(t)
print("Betweenness Analítico")
print(BAt)
print("\n")
print("Betweenness Simulado")
print(BSt)
print("\n")
print("Closeness Analítico")
print(CAt)
print("\n")
print("Closeness Simulado")
print(CSt)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 5.086538461538461), (3, 3.304505494505493), (7, 2.495329670329671), (11, 2.481648351648353), (8, 2.476428571428571), (9, 2.469670329670329), (2, 2.4642857142857135), (14, 2.4537912087912077), (13, 1.6283516483516478), (12, 1.6150549450549445), (5, 1.5659890109890102), (10, 0.7828021978021978), (1, 0.7558241758241754), (0, 0.7465934065934065), (4, 0.707802197802198)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526

Vamos dar uma verificada no comportamento desses testes para diferentes posições, analisando as extremidades, quando se tratando de betweenness.

In [37]:
allListsB = [BSa, BSb,  BSc, BSd, BSe, BSf,  BSh,  BSt]

In [38]:
for i in allListsB:
    print('Primeiro elemento é o nó: {0}'.format(i[0][0]))
    print('Segundo elemento é o nó: {0}'.format(i[1][0]))
    print('Penultimo elemento é o nó: {0}'.format(i[len(i)-2][0]))
    print('Ultimo elemento é o nó: {0}'.format(i[len(i)-1][0]))
    print('\n')

Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 0
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 7
Penultimo elemento é o nó: 10
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 0
Ultimo elemento é o nó: 4




In [39]:
allListsC = [CSa, CSb,  CSc, CSd, CSe, CSf,  CSh,  CSt]

In [40]:
for i in allListsC:
    print('Primeiro elemento é o nó: {0}'.format(i[0][0]))
    print('Segundo elemento é o nó: {0}'.format(i[1][0]))
    print('Penultimo elemento é o nó: {0}'.format(i[len(i)-2][0]))
    print('Ultimo elemento é o nó: {0}'.format(i[len(i)-1][0]))
    print('\n')

Primeiro elemento é o nó: 6
Segundo elemento é o nó: 2
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 2
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 7
Segundo elemento é o nó: 9
Penultimo elemento é o nó: 5
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 0
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 7
Segundo elemento é o nó: 9
Penultimo elemento é o nó: 5
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 13
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 13
Penultimo elemento é o nó: 5
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 4
Segundo elemento é o nó: 5
Penultimo elemento é o nó: 14
Ultimo elemento é o nó: 9




As divergências entre posições de importâncias das familias do período renascentista tende a ser mais estável trocando o algoritmo de locomoção de algo de um nó ao outro quando o indicador que determina a importância é o betweenness, as variações acontecem de forma mais aguda quando o fator determinante é o closeness.